# Convert IMD Gridded Data from Binary format to GeoTiff

We will use the open-source [imdlib](https://pypi.org/project/imdlib/) library to read the bindary files.

Install using PIP. Even if you are using conda, you can use PIP in Anaconda Prompt
```
pip install imdlib
```

Download the 0.25 x 0.25 degree gridded data from [IMD Climate Prediction Group Website](https://www.imdpune.gov.in/Clim_Pred_LRF_New/Grided_Data_Download.html)

This code assumes you have the downloaded files in `<home folder>/Downloads/imd`. Each year's `.grd` file located in this directory.

```
Downloads
|
|-- imd
     |
     |-- Rainfall_ind2019_rfp25.grd
     |-- Rainfall_ind2018_rfp25.grd
     |-- Rainfall_ind2017_rfp25.grd
     |-- ..
```

In [70]:
import calendar
import os
import imdlib as imd
import pandas as pd
import numpy as np
import rasterio
from rasterio.transform import Affine

The `imd.open_data()` method reads the binary file as a numpy array. The array needs to be transposed and flipped upside-down for it to be in correct orientation.

The parameters for affine transform are derived from the [dataset documentation](http://imdpune.gov.in/Clim_Pred_LRF_New/Grided_Data_Download.html).

In [74]:
def read_convert_write(year, output_filepath):
    variable = 'rain'
    file_format = ''
    data = imd.open_data(variable, year, year, file_format, imd_dir)
    transform = Affine(0.25, 0, 66.375, 0, -0.25, 38.625)
    # Transport array and flip the rows upside down
    output = data.data.transpose(0, 2 ,1)[:, ::-1, :]
    if calendar.isleap(year):
        count = 366
    else:
        count = 365
    dataset = rasterio.open(output_filepath, 'w', 
                                driver='GTiff',
                                height=129,
                                width=135,
                                count=count,
                                nodata=None,
                                dtype=np.float64,
                                crs='+proj=latlong',
                                transform=transform)
    dataset.write(output)
    dataset.close()

In [75]:
home_dir = os.path.join(os.path.expanduser('~'))
imd_dir = os.path.join(home_dir, 'Downloads', 'imd')

start_year = 2010
end_year = 2019
for year in range(start_year, end_year+1):
    output_file = '{}.tif'.format(year)
    output_filepath = os.path.join(imd_dir, output_file)
    read_convert_write(year, output_filepath)